In [ ]:
# Include DomainML src in module path
import sys
import os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Bypass issues with invoking notebook with server arguments
sys.argv.clear()
sys.argv.append("")

In [ ]:
import tensorflow as tf
import random

tensorflow_seed = 7796
random_seed = 82379498237

tf.random.set_seed(tensorflow_seed)
random.seed(random_seed)

# Load sequences

In [ ]:
from src.features import preprocessing
from src.features.preprocessing import mimic

preprocessor_config = mimic.MimicPreprocessorConfig()
preprocessor_config.prediction_column = "level_0"
preprocessor_config.sequence_column_name = "level_all"

sequences_df = preprocessing.MimicPreprocessor(preprocessor_config).load_data()

# Collect sequence metadata

In [ ]:
from src.features import sequences
from src.features.sequences import transformer

sequence_column_name = preprocessor_config.sequence_column_name

transformer_config = sequences.SequenceConfig()
transformer_config.x_sequence_column_name = "level_0"
transformer_config.y_sequence_column_name = "level_3"
transformer_config.predict_full_y_sequence_wide = True

transformer = transformer.NextPartialSequenceTransformerFromDataframe(transformer_config)

metadata = transformer.collect_metadata(sequences_df, sequence_column_name)

# Create dataset

In [ ]:
# We cannot use sequences.generate_train or sequences.generate_test because they internally use sequences.load_sequence_transformer, which will load the incorrect transformer in our case
def generate(for_train):
    train_sequences, test_sequences = transformer._split_train_test(sequences_df, sequence_column_name)
    relevant_sequences = train_sequences if for_train else test_sequences

    for sequence in relevant_sequences:
        split_sequences = transformer._split_sequence(sequence)
        for split_sequence in split_sequences:
            transformer._translate_and_pad(split_sequence, metadata)
            yield split_sequence.x_vecs_stacked, split_sequence.y_vec

def generate_train():
    return generate(for_train=True)

def generate_test():
    return generate(for_train=False)

In [ ]:
dataset_shuffle_buffer = 1000
dataset_shuffle_seed = 12345
batch_size = 32

train_dataset = (tf.data.Dataset.from_generator(generate_train, output_types=(tf.float32, tf.float32))
.shuffle(dataset_shuffle_buffer, dataset_shuffle_seed, reshuffle_each_iteration=True)
.batch(batch_size)
.prefetch(tf.data.experimental.AUTOTUNE))

test_dataset = (tf.data.Dataset.from_generator(generate_test,
output_types=(tf.float32, tf.float32))
.batch(batch_size)
.prefetch(tf.data.experimental.AUTOTUNE))

# Load model

In [ ]:
from src.features import knowledge

hierarchy_df = preprocessing.ICD9HierarchyPreprocessor(preprocessor_config).load_data()
hierarchy = knowledge.HierarchyKnowledge(knowledge.KnowledgeConfig())
hierarchy.build_hierarchy_from_df(hierarchy_df, metadata.x_vocab)

from src.training import models

model = models.GramModel()

# Build model

In [ ]:
model.build(metadata, hierarchy)

# Train model

In [ ]:
multilabel_classification = False
n_epochs = 1

model.train_dataset(train_dataset, test_dataset, multilabel_classification, n_epochs)